In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [2]:
# ONE Make an RDD of passwords
pw = sc.textFile('../data/passwords.csv')
pw.collect()
pw.take(5)

['adimabi9', 'ne14a69r', 'Xavier2791', 'INDele19SHEL', '4200bpcC']

In [3]:
def subs(s):
    l = [(s,s[i:j+1].lower()) for i in range(len(s)) for j in range(i,len(s))]
    return l

In [4]:
# TWO Map of tuples (password, [list of all possible substrings])
pwt = pw.map(lambda x:(x,subs(x)))
pwt.take(1)

[('adimabi9',
  [('adimabi9', 'a'),
   ('adimabi9', 'ad'),
   ('adimabi9', 'adi'),
   ('adimabi9', 'adim'),
   ('adimabi9', 'adima'),
   ('adimabi9', 'adimab'),
   ('adimabi9', 'adimabi'),
   ('adimabi9', 'adimabi9'),
   ('adimabi9', 'd'),
   ('adimabi9', 'di'),
   ('adimabi9', 'dim'),
   ('adimabi9', 'dima'),
   ('adimabi9', 'dimab'),
   ('adimabi9', 'dimabi'),
   ('adimabi9', 'dimabi9'),
   ('adimabi9', 'i'),
   ('adimabi9', 'im'),
   ('adimabi9', 'ima'),
   ('adimabi9', 'imab'),
   ('adimabi9', 'imabi'),
   ('adimabi9', 'imabi9'),
   ('adimabi9', 'm'),
   ('adimabi9', 'ma'),
   ('adimabi9', 'mab'),
   ('adimabi9', 'mabi'),
   ('adimabi9', 'mabi9'),
   ('adimabi9', 'a'),
   ('adimabi9', 'ab'),
   ('adimabi9', 'abi'),
   ('adimabi9', 'abi9'),
   ('adimabi9', 'b'),
   ('adimabi9', 'bi'),
   ('adimabi9', 'bi9'),
   ('adimabi9', 'i'),
   ('adimabi9', 'i9'),
   ('adimabi9', '9')])]

In [5]:
def agg(x,y):
    if type(x) is list and type(y) is list: #if the two keys are lists, combine them
        return x+y
    if type(x) is list:  #if the first one is a list only, just append y
        x.append(y)
        return x
    else: #if neither is a list, create a list with both of them
        l = []
        l.append(x)
        l.append(y)
        return l
def map_count(x):
    if type(x[1]) is list:  #just get length of list and that is your "count"
        return (x[0],len(x[1]))
    else:
        return (x[0],1) #if there is no list, just output a '1'

In [13]:
# THREE - SEVEN Reduce by key to count passwords per string
#3 = filter
#4 = reduceByKey with agg function
#5 = mapping to count number of passwords with map_count
#6 = sortBy value, descending
#7 = .take(20)
pw.flatMap(subs).filter(lambda x:len(x[1]) == 6).map(lambda x:(x[1],x[0])).\
reduceByKey(agg).map(map_count).sortBy(lambda x:-x[1]).take(20)

[('sheldu', 69),
 ('heldus', 69),
 ('eldus1', 13),
 ('passwo', 10),
 ('asswor', 10),
 ('hazard', 10),
 ('ssword', 9),
 ('123456', 8),
 ('eather', 7),
 ('eldus2', 7),
 ('search', 6),
 ('ssheld', 6),
 ('weathe', 6),
 ('monste', 6),
 ('onster', 6),
 ('234567', 5),
 ('ldus12', 5),
 ('disast', 5),
 ('isaste', 5),
 ('er1234', 5)]

In [7]:
# FOUR - SEVEN Reduce by key to count passwords per string
# ""
pw.flatMap(subs).filter(lambda x:len(x[1]) == 10).map(lambda x:(x[1],x[0])).\
reduceByKey(agg).map(map_count).sortBy(lambda x:-x[1]).take(20)

[('sheldus201', 5),
 ('mitigation', 4),
 ('sheldus123', 3),
 ('minustwo2?', 3),
 ('zkcsheldus', 3),
 ('watermelon', 3),
 ('heldus2017', 3),
 ('33theresa2', 2),
 ('3theresa27', 2),
 ('!lhy324123', 2),
 ('shelduspas', 2),
 ('helduspass', 2),
 ('ruth13viol', 2),
 ('uth13violi', 2),
 ('th13violin', 2),
 ('hazarddata', 2),
 ('azarddata2', 2),
 ('zarddata20', 2),
 ('arddata201', 2),
 ('rddata2014', 2)]

Length 10 gives 'mitigation', 'watermelon', and 'minustwo2?' as some of the most used substrings of passwords.

In [8]:
# FOUR - SEVEN Reduce by key to count passwords per string
# ""
pw.flatMap(subs).filter(lambda x:len(x[1]) == 8).map(lambda x:(x[1],x[0])).\
reduceByKey(agg).map(map_count).sortBy(lambda x:-x[1]).take(20)

[('sheldus1', 13),
 ('password', 9),
 ('sheldus2', 7),
 ('ssheldus', 6),
 ('heldus12', 5),
 ('heldus20', 5),
 ('eldus201', 5),
 ('adimabi9', 4),
 ('research', 4),
 ('disaster', 4),
 ('jung6397', 4),
 ('sunshine', 4),
 ('12345678', 4),
 ('mitigati', 4),
 ('itigatio', 4),
 ('tigation', 4),
 ('ne14a69r', 3),
 ('eldus123', 3),
 ('sheldusp', 3),
 ('helduspa', 3)]

Length 8 gives 'sheldus1' and 'password' as the most common types of substrings in passwords.